# Model 2: Line-by-Line Sequence

In [1]:
"""

Jack and Jill went up the hill To fetch a pail 
of water Jack fell down and broke his crown And 
Jill came tumbling after



X,                                y
_, _, _, _, _, Jack,            and 
_, _, _, _, Jack, and,          Jill
_, _, _, Jack, and, Jill,       went
_, _, Jack, and, Jill, went,     up 
_, Jack, and, Jill, went, up,    the
Jack, and, Jill, went, up, the,  hill

"""

'\n\nJack and Jill went up the hill To fetch a pail \nof water Jack fell down and broke his crown And \nJill came tumbling after\n\n\n\nX,                                y\n_, _, _, _, _, Jack,            and \n_, _, _, _, Jack, and,          Jill\n_, _, _, Jack, and, Jill,       went\n_, _, Jack, and, Jill, went,     up \n_, Jack, and, Jill, went, up,    the\nJack, and, Jill, went, up, the,  hill\n\n'

In [2]:
from numpy import array 
from keras.preprocessing.text import Tokenizer 
from keras.utils import to_categorical 
from keras.utils.vis_utils import plot_model 
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import LSTM 
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [3]:
# generate a sequence from a language model 
def generate_seq(model, tokenizer, max_length, seed_text, n_words): 
    in_text = seed_text 
    # generate a fixed number of words 
    for _ in range(n_words): 
        # encode the text as integer 
        encoded = tokenizer.texts_to_sequences([in_text])[0] 
        # pre-pad sequences to a fixed length 
        encoded = pad_sequences([encoded], maxlen=max_length, padding='pre') 
        # predict probabilities for each word 
        yhat = model.predict_classes(encoded, verbose=0) 
        # map predicted word index to word 
        out_word = '' 
        for word, index in tokenizer.word_index.items(): 
            if index == yhat: 
                out_word = word 
                break 
        # append to input 
        in_text += ' ' + out_word 
    return in_text


In [5]:
# define the model 
def define_model(vocab_size, max_length): 
    model = Sequential() 
    model.add(Embedding(vocab_size, 10, input_length=max_length-1)) 
    model.add(LSTM(50)) 
    model.add(Dense(vocab_size, activation='softmax')) 
    # compile network 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
    # summarize defined  
    model.summary() 
    plot_model(model, to_file='word_model2_linebyline.png', show_shapes=True) 
    return model


In [6]:
# source text 
data = """ Jack and Jill went up the hill\n 
To fetch a pail of water\n Jack fell down and broke his crown\n 
And Jill came tumbling after\n """

In [7]:
# prepare the tokenizer on the source text 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts([data]) 
# determine the vocabulary size 
vocab_size = len(tokenizer.word_index) + 1 
print('Vocabulary Size: %d' % vocab_size) 

Vocabulary Size: 22


In [8]:
# create line-based sequences 
sequences = list() 
for line in data.split('\n'): 
    encoded = tokenizer.texts_to_sequences([line])[0] 
    for i in range(1, len(encoded)): 
        sequence = encoded[:i+1] 
        sequences.append(sequence) 
print('Total Sequences: %d' % len(sequences))


Total Sequences: 21


In [9]:
sequences,len(sequences)

([[2, 1],
  [2, 1, 3],
  [2, 1, 3, 4],
  [2, 1, 3, 4, 5],
  [2, 1, 3, 4, 5, 6],
  [2, 1, 3, 4, 5, 6, 7],
  [8, 9],
  [8, 9, 10],
  [8, 9, 10, 11],
  [8, 9, 10, 11, 12],
  [8, 9, 10, 11, 12, 13],
  [2, 14],
  [2, 14, 15],
  [2, 14, 15, 1],
  [2, 14, 15, 1, 16],
  [2, 14, 15, 1, 16, 17],
  [2, 14, 15, 1, 16, 17, 18],
  [1, 3],
  [1, 3, 19],
  [1, 3, 19, 20],
  [1, 3, 19, 20, 21]],
 21)

In [10]:
# pad input sequences 
max_length = max([len(seq) for seq in sequences]) 
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre') 
print('Max Sequence Length: %d' % max_length)


Max Sequence Length: 7


In [11]:
# split into input and output elements 
sequences = array(sequences) 
X, y = sequences[:,:-1],sequences[:,-1] 
y = to_categorical(y, num_classes=vocab_size)


In [12]:
X,y

(array([[ 0,  0,  0,  0,  0,  2],
        [ 0,  0,  0,  0,  2,  1],
        [ 0,  0,  0,  2,  1,  3],
        [ 0,  0,  2,  1,  3,  4],
        [ 0,  2,  1,  3,  4,  5],
        [ 2,  1,  3,  4,  5,  6],
        [ 0,  0,  0,  0,  0,  8],
        [ 0,  0,  0,  0,  8,  9],
        [ 0,  0,  0,  8,  9, 10],
        [ 0,  0,  8,  9, 10, 11],
        [ 0,  8,  9, 10, 11, 12],
        [ 0,  0,  0,  0,  0,  2],
        [ 0,  0,  0,  0,  2, 14],
        [ 0,  0,  0,  2, 14, 15],
        [ 0,  0,  2, 14, 15,  1],
        [ 0,  2, 14, 15,  1, 16],
        [ 2, 14, 15,  1, 16, 17],
        [ 0,  0,  0,  0,  0,  1],
        [ 0,  0,  0,  0,  1,  3],
        [ 0,  0,  0,  1,  3, 19],
        [ 0,  0,  1,  3, 19, 20]]),
 array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [13]:
# define model 
model = define_model(vocab_size, max_length) 









_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 6, 10)             220       
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_1 (Dense)              (None, 22)                1122      
Total params: 13,542
Trainable params: 13,542
Non-trainable params: 0
_________________________________________________________________


In [14]:
# fit network 
model.fit(X, y, epochs=500, verbose=2) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/500
 - 2s - loss: 3.0909 - acc: 0.0952
Epoch 2/500
 - 0s - loss: 3.0899 - acc: 0.1905
Epoch 3/500
 - 0s - loss: 3.0886 - acc: 0.1905
Epoch 4/500
 - 0s - loss: 3.0871 - acc: 0.1905
Epoch 5/500
 - 0s - loss: 3.0857 - acc: 0.1429
Epoch 6/500
 - 0s - loss: 3.0843 - acc: 0.1429
Epoch 7/500
 - 0s - loss: 3.0828 - acc: 0.0952
Epoch 8/500
 - 0s - loss: 3.0813 - acc: 0.0952
Epoch 9/500
 - 0s - loss: 3.0797 - acc: 0.0952
Epoch 10/500
 - 0s - loss: 3.0780 - acc: 0.0952
Epoch 11/500
 - 0s - loss: 3.0763 - acc: 0.0952
Epoch 12/500
 - 0s - loss: 3.0746 - acc: 0.0952
Epoch 13/500
 - 0s - loss: 3.0727 - acc: 0.0952
Epoch 14/500
 - 0s - loss: 3.0708 - acc: 0.0952
Epoch 15/500
 - 0s - loss: 3.0687 - acc: 0.0952
Epoch 16/500
 - 0s - loss: 3.0666 - acc: 0.0952
Epoch 17/500
 - 0s - loss: 3.0643 - acc: 0.0952
Epoch 18/500
 - 0s - loss: 3.0620 - acc: 0.0952
Epoch 19/500
 - 0s - loss: 3.0595 - acc: 0.0952
Ep

Epoch 159/500
 - 0s - loss: 0.5544 - acc: 0.9048
Epoch 160/500
 - 0s - loss: 0.5494 - acc: 0.9048
Epoch 161/500
 - 0s - loss: 0.5442 - acc: 0.9048
Epoch 162/500
 - 0s - loss: 0.5387 - acc: 0.9048
Epoch 163/500
 - 0s - loss: 0.5324 - acc: 0.9048
Epoch 164/500
 - 0s - loss: 0.5273 - acc: 0.9048
Epoch 165/500
 - 0s - loss: 0.5230 - acc: 0.9048
Epoch 166/500
 - 0s - loss: 0.5176 - acc: 0.9048
Epoch 167/500
 - 0s - loss: 0.5122 - acc: 0.9048
Epoch 168/500
 - 0s - loss: 0.5078 - acc: 0.9048
Epoch 169/500
 - 0s - loss: 0.5031 - acc: 0.9048
Epoch 170/500
 - 0s - loss: 0.4982 - acc: 0.9048
Epoch 171/500
 - 0s - loss: 0.4935 - acc: 0.9048
Epoch 172/500
 - 0s - loss: 0.4892 - acc: 0.9048
Epoch 173/500
 - 0s - loss: 0.4847 - acc: 0.9048
Epoch 174/500
 - 0s - loss: 0.4803 - acc: 0.9048
Epoch 175/500
 - 0s - loss: 0.4760 - acc: 0.9048
Epoch 176/500
 - 0s - loss: 0.4719 - acc: 0.9048
Epoch 177/500
 - 0s - loss: 0.4679 - acc: 0.9048
Epoch 178/500
 - 0s - loss: 0.4637 - acc: 0.9048
Epoch 179/500
 - 0s 

Epoch 327/500
 - 0s - loss: 0.1765 - acc: 0.9524
Epoch 328/500
 - 0s - loss: 0.1757 - acc: 0.9524
Epoch 329/500
 - 0s - loss: 0.1748 - acc: 0.9524
Epoch 330/500
 - 0s - loss: 0.1740 - acc: 0.9524
Epoch 331/500
 - 0s - loss: 0.1732 - acc: 0.9524
Epoch 332/500
 - 0s - loss: 0.1725 - acc: 0.9524
Epoch 333/500
 - 0s - loss: 0.1718 - acc: 0.9524
Epoch 334/500
 - 0s - loss: 0.1710 - acc: 0.9524
Epoch 335/500
 - 0s - loss: 0.1702 - acc: 0.9524
Epoch 336/500
 - 0s - loss: 0.1694 - acc: 0.9524
Epoch 337/500
 - 0s - loss: 0.1686 - acc: 0.9524
Epoch 338/500
 - 0s - loss: 0.1678 - acc: 0.9524
Epoch 339/500
 - 0s - loss: 0.1671 - acc: 0.9524
Epoch 340/500
 - 0s - loss: 0.1664 - acc: 0.9524
Epoch 341/500
 - 0s - loss: 0.1657 - acc: 0.9524
Epoch 342/500
 - 0s - loss: 0.1650 - acc: 0.9524
Epoch 343/500
 - 0s - loss: 0.1642 - acc: 0.9524
Epoch 344/500
 - 0s - loss: 0.1635 - acc: 0.9524
Epoch 345/500
 - 0s - loss: 0.1628 - acc: 0.9524
Epoch 346/500
 - 0s - loss: 0.1622 - acc: 0.9524
Epoch 347/500
 - 0s 

Epoch 495/500
 - 0s - loss: 0.1048 - acc: 0.9524
Epoch 496/500
 - 0s - loss: 0.1046 - acc: 0.9524
Epoch 497/500
 - 0s - loss: 0.1044 - acc: 0.9524
Epoch 498/500
 - 0s - loss: 0.1042 - acc: 0.9524
Epoch 499/500
 - 0s - loss: 0.1040 - acc: 0.9524
Epoch 500/500
 - 0s - loss: 0.1038 - acc: 0.9524


In [15]:
# evaluate model 
print(generate_seq(model, tokenizer, max_length-1, 'Jack', 4)) 
print(generate_seq(model, tokenizer, max_length-1, 'Jill', 4))

Jack fell down and broke
Jill jill came tumbling after
